In [3]:
import gc

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

In [4]:
import torch
from transformers import MBartForConditionalGeneration, MBart50Tokenizer
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

def get_model(model_path="./mbart-large-51-mans-ru-v3-full-finetuneepoch_4/"):
    tok_path = './mbart-large-51-mans-raw/'
    tokenizer = MBart50Tokenizer.from_pretrained(tok_path)
    old_len = len(tokenizer)
    tokenizer.lang_code_to_id['mans_XX'] = old_len-1
    tokenizer.id_to_lang_code[old_len-1] = 'mans_XX'
    tokenizer.fairseq_tokens_to_ids["<mask>"] = len(tokenizer.sp_model) + len(tokenizer.lang_code_to_id) + tokenizer.fairseq_offset
    tokenizer.fairseq_tokens_to_ids.update(tokenizer.lang_code_to_id)
    tokenizer.fairseq_ids_to_tokens = {v: k for k, v in tokenizer.fairseq_tokens_to_ids.items()}
    if 'mans_XX' not in tokenizer._additional_special_tokens:
        tokenizer._additional_special_tokens.append('mans_XX')

    tokenizer.src_lang = "mans_XX"
    tokenizer.tgt_lang = "ru_RU"
    
    
    model = MBartForConditionalGeneration.from_pretrained(model_path,
                                                        torch_dtype=torch.bfloat16,
                                                        #attn_implementation="flash_attention_2", #https://huggingface.co/docs/transformers/perf_infer_gpu_one#combine-optimizations
                                                        #WORKS ONLY ON AMPERS GPUS
                                                        load_in_8bit=True,
                                                         )
    print(model.device)
    
    return model, tokenizer

model, tokenizer = get_model()

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


cuda:0


In [5]:
# model = model.to_bettertransformer()

model = torch.compile(model) #https://huggingface.co/docs/transformers/perf_torch_compile#v100-batch-size-1
# https://habr.com/ru/companies/wunderfund/articles/820721/

In [27]:
def translate(text, src='mans_XX', trg='ru_RU', max_length=200, num_beams=5, repetition_penalty=5.0, **kwargs):
    tokenizer.src_lang = src
    encoded = tokenizer(text, return_tensors="pt")
    
    # # enable FlashAttention
    # with torch.backends.cuda.sdp_kernel(enable_flash=True, enable_math=False, enable_mem_efficient=False):
    ### works ONLY ON AMPER GPUS
    
    generated_tokens = model.generate(
        **encoded.to(model.device),
        forced_bos_token_id=tokenizer.lang_code_to_id[trg], 
        max_length=max_length, 
        num_beams=num_beams,
        repetition_penalty=repetition_penalty,
        # early_stopping=True,
    )
        
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]


def translate_batch(texts, src='mans_XX', trg='ru_RU', max_length=200, num_beams=5, repetition_penalty=5.0,
                    batch_size=128, **kwargs):
    tokenizer.src_lang = src
    all_translations = []

    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i+batch_size]
        encoded = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True)
        
        # # enable FlashAttention
        # with torch.backends.cuda.sdp_kernel(enable_flash=True, enable_math=False, enable_mem_efficient=False):
        ### works ONLY ON AMPER GPUS
        
        generated_tokens = model.generate(
            **{k: v.to(model.device) for k, v in encoded.items()},
            forced_bos_token_id=tokenizer.lang_code_to_id[trg],
            max_length=max_length,
            num_beams=num_beams,
            repetition_penalty=repetition_penalty,
            # early_stopping=True,
        )
        
        batch_translations = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        all_translations.extend(batch_translations)
    
    return all_translations

def translate_dataframe(df, text_column, src='mans_XX', trg='ru_RU', **kwargs):
    texts = df[text_column].tolist()
    domains = df['domain'].values
    translations = translate_batch(texts, src=src, trg=trg, **kwargs)
    return pd.DataFrame({'original': texts, 'ru_target': df['ru'].values, 'translation_model': translations, 'domain': domains})

In [34]:
import evaluate

chrf = evaluate.load("chrf")
bleu = evaluate.load("bleu")

def get_simple_metrics(data: pd.DataFrame, preds_column: str,
                       original_column: str, output_file: str):
    
    chrf__ = chrf.compute(
        predictions=data[preds_column].values,
        references=data[original_column].values,
        word_order=2,
    )["score"]
    bleu__ = bleu.compute(
        predictions=data[preds_column].values, references=data[original_column].values
    )["bleu"]
    metrics = pd.DataFrame(
        {"chrf": [chrf__], "bleu": [bleu__]}
    )
    print(output_file, f"bleu {bleu__}", f"chrf {chrf__}")
    metrics.to_csv(output_file, index=False)

In [35]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

model_names = ["mbart-large-51-mans-ru-v3-full-finetuneepoch_4",]

test = pd.read_csv('./datasets/test_set_clean.csv', index_col=0)
print(f"test size: {test.shape[0]}")

def run_model_test(model_name, preds_column, original_column):
    translated_test = translate_dataframe(test, text_column="mans")
    
    #save results
    translated_test.to_csv(f'./metrics/test_predicted_{model_name}.csv')
        
        
    #compute metrics
    get_simple_metrics(translated_test, preds_column=preds_column,
                       original_column='ru_target',
                       output_file=f"./metrics/results_{model_name}.csv")


test size: 8842


In [36]:
run_model_test(model_names[0], preds_column='translation_model', original_column='ru')

100%|██████████| 70/70 [25:55<00:00, 22.22s/it]


./metrics/results_mbart-large-51-mans-ru-v3-full-finetuneepoch_4.csv bleu 0.16391773395264653 chrf 40.32924085019285


In [32]:
#simple test

sample = test.sample(1)
sample['mans'], sample['ru'], translate(sample['mans'].values[0])

(2078    Та̄н ты ма̄ тармыл ма̄к кӯщаиг о̄лэ̄гыт.
 Name: mans, dtype: object,
 2078    Они на этой земле настоящие есть хозяева.
 Name: ru, dtype: object,
 'Они на этой земле являются главными.')

In [21]:
chrf = evaluate.load("chrf")
bleu = evaluate.load("bleu")
    
print('chrf', chrf.compute(predictions=['В этом месяце их в город Ханты-Мансийск приглашали.'], references=['Ещё в этом месяце их в город Ханты-Мансийск по...'], word_order=2))

print('bleu', bleu.compute(predictions=['А̄ги Куринька ма̄щтыр.'], references=['А̄ги Кӯринька ва̄ранты.']))


chrf {'score': 74.87502323834276, 'char_order': 6, 'word_order': 2, 'beta': 2}
bleu {'bleu': 0.0, 'precisions': [0.5, 0.0, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 4, 'reference_length': 4}


# Черновик

In [12]:

bible_mansi_iona = pd.read_csv('./datasets/bible_mansi_iona.csv')
bible_mansi_iona.columns

Index(['source', 'target'], dtype='object')

In [13]:
bible_mansi_iona

,source,target
0,"Ионан, Амиттай пы̄гын, Тōрум Āсьныл тамле лāты...","И было слово Господне к Ионе, сыну Амафиину:"
1,Иона э̄лы лё̄ӈхын минас: Тōрум Āсьныл Таршишн ...,"И встал Иона, чтобы бежать в Фарсис от лица Го..."
2,Кāраплит рупитан хōтпат пилысьман ёхтувēсыт. К...,"И устрашились корабельщики, и взывали каждый к..."
3,Тāн тāнки халанылт потыртаӈкве патсыт: ”Халувт...,"И сказали друг другу: пойдем, бросим жребии, ч..."
4,"”Ам – еврей, – тав лāвыс, – ам Тōрум Āсь янытл...","И он сказал им: я Еврей, чту Господа Бога небе..."
5,"– Мāнавн наӈынтыл мāныр вāруӈкве, ся̄рысь яныг...","И сказали ему: что сделать нам с тобою, чтобы ..."
6,"Кāраплит рупитан хōтпат пāг товуӈкве патсыт, н...","Но эти люди начали усиленно грести, чтобы прис..."
7,"Иона āлмысаныл, таве ся̄рсин та вуськасасāныл ...","И взяли Иону и бросили его в море, и утихло мо..."
8,”Э̄лы лё̄ӈхын нёвумтэ̄н! Ниневия нампа яныг yс...,"встань, иди в Ниневию, город великий, и пропов..."
9,"Тōрум Āсь ся̄рсин яныг вōт кēтыс, ос ся̄рсит я...","Но Господь воздвиг на море крепкий ветер, и сд..."


In [6]:
sample = bible_mansi_iona.sample(1, random_state=42)
sample

,source,target
27,Ōс Ионан Тōрум Āсь лāтыӈ та суйтыс:,И было слово Господне к Ионе вторично:


In [14]:
ru, mans = medvezhie_pesni['target'].values[0], medvezhie_pesni['source'].values[0]

ru, mans

('Упрямыми руками медведицу-доченьку', 'Вориц катпа нюрум уй агикве')

In [ ]:
from tqdm import tqdm
tqdm.pandas()

medvezhie_pesni['predicted_mbart_3_epochs'] = medvezhie_pesni['target'].progress_apply(lambda x: translate(x))

 48%|████▊     | 2778/5810 [11:25<11:39,  4.33it/s]

In [15]:
bible_mansi_iona['predicted_mbart_3_epochs'];

In [8]:
medvezhie_pesni = pd.read_csv('./datasets/medvezhie_pesni.csv')

pesni_hulimsunt = pd.read_csv('./datasets/pesni_hulimsunt.csv')

# Computing Metrics

In [19]:
medvezhie_pesni.head(3)

,source,target,predicted_mbart_3_epochs
0,"Ионан, Амиттай пы̄гын, Тōрум Āсьныл тамле лāты...","И было слово Господне к Ионе, сыну Амафиину:",Тōрум Āсь Иоанн Амафиин пыге нупыл лāвыс:
1,Иона э̄лы лё̄ӈхын минас: Тōрум Āсьныл Таршишн ...,"И встал Иона, чтобы бежать в Фарсис от лица Го...",Иона Фарсисн Тōрум Āсь нупыл оюӈкве но̄х-ква̄л...
2,Кāраплит рупитан хōтпат пилысьман ёхтувēсыт. К...,"И устрашились корабельщики, и взывали каждый к...","Ань юи-па̄лт, матхāн хум хо̄тпат хот-рохтысыт,..."


In [ ]:
medvezhie_pesni.to_csv('./metrics/medvezhie_pesni_preds.csv')